In [5]:
import psycopg2
import json
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

number_of_results = 5

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Function to perform the search
def search(query, model, db_config):
    # Encode the query
    query_embedding = model.encode([query])[0]
    query_embedding = [float(value) for value in query_embedding]

    
    # Connect to the database
    with psycopg2.connect(**db_config) as conn:
        c = conn.cursor()

        # Search for the top projects by cosine similarity
        c.execute("""
            SELECT project_id, chunk, embedding <-> %s::VECTOR AS distance
            FROM embeddings
            ORDER BY distance DESC
            LIMIT %s;
        """, (list(query_embedding), number_of_results))
        
        results = c.fetchall()
    
    return results

# Initialize the model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Define the database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}



In [6]:
#convert this notebook to a python script
!jupyter nbconvert --to script search.ipynb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[NbConvertApp] Converting notebook search.ipynb to script
[NbConvertApp] Writing 1643 bytes to search.py


In [7]:
# Define the query
query = "Main challengues for Climate Smart Agriculture"

# Perform the search
results = search(query, model, db_config)
# Print the results
for project_id, chunk, distance in results:
    project = next((p for p in projects if p["ids"] == [project_id]), None)
    print(f"Project ID: {project_id}")
    print(f"Project title: {project['title']}")
    print(f"Project abstract: {project['abstract']}")
    print(f"Relevant snippet: {chunk}")
    print(f"Distance: {distance}")
    print("\n")


Project ID: P151072
Project title: India - National Agriculture Higher Education Project : indigenous peoples plan : Indigenous peoples policy framework
Project abstract: The objective of the National Agriculture Higher Education Project for India is: ‘to support participating Agricultural Universities and ICAR in providing more relevant and higher quality education to agriculture university students.’ ICAR has prepared this Equity Action Plan (EAP) or Indigenous People’s Policy Framework (IPPF) which addresses issues of gender equality and social inclusion with special attention to the needs of the Scheduled Tribe and the Scheduled Caste students and faculty members fulfilling the requirements of OP 4.10. This EAP/IPPF has been prepared using mostly qualitative research methodologies, including an online survey with the primary stakeholder - students and faculties from various social backgrounds, including ST and SC groups. This will be discussed and finalized with intensive stakehold